# Meshing a Cube with snappyHexMesh

This notebook shows how to mesh a simple cube using snappyHexMesh in Flow360. This is a very simple geometry to show the integration of snappyHexMesh in flow360.

## 1. Create Project from geometry
- Load Python libraries and Flow360 client. If you use environment variables or tokens, initialize them here so later API calls can authenticate.
- Project is created from geometry CAD file of the cube.

In [1]:
import flow360 as fl
from flow360.examples import Cube

Cube.get_files()

solver_version = "snappyHex-25.8.7"
project = fl.Project.from_geometry(
    Cube.geometry,
    name='Meshing Cube with Snappy',
    length_unit='m',
    solver_version=solver_version)

geo = project.geometry
geo.show_available_groupings(verbose_mode=True)
geo.group_faces_by_tag("faceId")

Output()

[20:19:26] INFO: Geometry successfully submitted:                                                                  
                   type   = Geometry                                                                               
                   name   = Meshing Cube with Snappy                                                               
                   id     = geo-ff48cc97-5340-4b16-b3f9-7edab07c2406                                               
                   status = uploaded                                                                               
           

           INFO: Waiting for geometry to be processed.

Output()

[20:19:56] INFO:  >> Available attribute tags for grouping **faces**:

...

           INFO:  >> Available attribute tags for grouping **bodies**:

           INFO:     >> Tag '1': groupByFile. Grouping with this tag results in:

           INFO:         >> [0]: cube.stl

           INFO:            IDs: ['cube.stl']

           INFO: Regrouping face entities under `faceId` tag (previous `groupByBodyId`).

## 2. Define meshing parameters
The global meshing parameters are defined with the modular meshing workflow which allows to choose the mesher for surface and volume meshes. In this case, we'll be using the snappy integration as surface meshing and the beta mesher for the volume mesh.

In [2]:
with fl.SI_unit_system:
    farfield = fl.AutomatedFarfield()
    meshing_params=fl.ModularMeshingWorkflow(
        surface_meshing=fl.SnappySurfaceMeshingParams(
            defaults=fl.SnappySurfaceMeshingDefaults(
                min_spacing=5*fl.u.mm,
                max_spacing=100*fl.u.mm,
                gap_resolution=1*fl.u.mm
            )
        ),
        volume_meshing=fl.BetaVolumeMeshingParams(
            defaults=fl.BetaVolumeMeshingDefaults(
                boundary_layer_first_layer_thickness=0.1*fl.u.mm
            )
        ),
        zones=[farfield]
    )

           INFO: using: SI unit system for unit inference.

## 3. Define SimulationParams
- The simulation parameters are defined in the python class fl.SimulationParams()
- The meshing configuration defined previously is used here.

In [3]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        meshing=meshing_params,
        operating_condition=fl.AerospaceCondition(
            velocity_magnitude=10*fl.u.m/fl.u.s
        ),
        time_stepping=fl.Steady(),
        models=[
            fl.Wall(
                surfaces=geo['*']
            ),
            fl.Freestream(
                surfaces=[farfield.farfield]
            )
        ],
        reference_geometry=fl.ReferenceGeometry()
    )

           INFO: using: SI unit system for unit inference.

## 4. Run Case

In [4]:
case = project.run_case(params=params, name="Case of tutorial BETDisk from Python")

           INFO: using: SI unit system for unit inference.

[20:19:58] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Case of tutorial BETDisk from Python                                                   
                   id     = case-4ee479ce-e8a8-4f4b-9faf-7312c09425b9                                              
                   status = pending                                                                                
           